In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import os
srcpath = os.path.join(os.path.abspath(os.path.dirname("../"),))
sys.path.append(srcpath)
from src.readGraph import readgraph
import cupy as cnp
import numpy as np 
import time
import pandas as pd
from metrics import SE,NP1,NP2,MA,EC
import numba 
numba.set_num_threads(32)

In [2]:
graphs = ['dwt_72', 'lesmis', 'can_96', 'rajat11', 'jazz', 'visbrazil',
       'grid17', 'mesh3e1', 'netscience', 'dwt_419','cluster', 'qh882',
       'price_1000', 'dwt_1005', 'cage8', 'btree9', 'bcsstk09',
       '1138_bus', 'qh1484','clusterHiera' , 'bcspwr07', 'fidapex6', 'block_2000',
       'sierpinski3d', 'lp_ship04l', 'data', '3elt', 'USPowerGrid',
       'add32', 'ca-GrQc', 'commanche', 'EVA', 'bcsstk33',
       'whitaker3', 'crack', 'fe_4elt2', 'fidapex11','bcsstk31', 'bcsstk32', 'finan512', 'luxembourg', 'fe_ocean',
        'com-dblp.ungraph',
        'com-amazon.ungraph',
        'roadNet-PA',
        'com-youtube.ungraph',
        'roadNet-TX',
        'roadNet-CA',
        'com-orkut.ungraph',
        'com-lj.ungraph']

In [3]:
idx_a  = 37 # index_can_be_processed_by_all
idx_l = 42 # index_very_large_start

In [4]:
dirs = ["t-FDP_approx/Exact","t-FDP_approx/BH","t-FDP_approx/RVS","t-FDP_approx/ibFFT_CPU","t-FDP_approx/ibFFT_GPU"]
methods = ["Exact","BH","RVS","ibFFT_CPU","ibFFT_GPU"]
        

In [5]:
repetitions = 5
values = np.zeros((idx_l,len(methods)))
graphidx = 0
# for graphname in (graphs): # all graphs
for graphname in (graphs[:idx_l]):
    filename = "../data/" + graphname + ".mtx"
    graph,edgesrc,edgetgt,_ = readgraph(filename,False)
    for i in range(len(methods)):
        sums = 0
        for k in range(repetitions):
            posfile = "../layout_results/" + dirs[i] + "/" + graphname + "." + methods[i] + "_" + str(k)
            if os.path.isfile(posfile):
                if methods[i] == "DRGraph":
                    pos = np.loadtxt(posfile,delimiter=' ',skiprows=1)
                    ## The first line of DRGraph's output is the description of the number of rows and columns
                else:
                    pos = np.loadtxt(posfile,delimiter=' ')
                se,se_norm = SE(pos, edgesrc, edgetgt)
            else:
                se = np.nan
                se_norm = np.nan
            sums += se_norm     
        mean = sums/repetitions
        values[graphidx,i] = mean
    graphidx+=1
df_se = pd.DataFrame(index = graphs[:idx_l])
for i in range(len(methods)):
    df_se[methods[i]] = values[:,i]
df_se.to_csv("./results/Approx_SE.csv")

In [6]:
df_se[:idx_a].mean()

Exact        0.078894
BH           0.078986
RVS          0.105188
ibFFT-CPU    0.078702
ibFFT-GPU    0.078358
dtype: float64

In [7]:
from numba_kdtree import KDTree

repetitions = 5
values = np.zeros((idx_l,len(methods)))
graphidx = 0
# for graphname in (graphs): # all graphs
for graphname in (graphs[:idx_l]):
    filename = "../data/" + graphname + ".mtx"
    graph,edgesrc,edgetgt,_ = readgraph(filename,False)
    for i in range(len(methods)):
        sums = 0
        for k in range(repetitions):
            posfile = "../layout_results/" + dirs[i] + "/" + graphname + "." + methods[i] + "_" + str(k)
            if os.path.isfile(posfile):
                if methods[i] == "DRGraph":
                    pos = np.loadtxt(posfile,delimiter=' ',skiprows=1)
                    ## The first line of DRGraph's output is the description of the number of rows and columns
                else:
                    pos = np.loadtxt(posfile,delimiter=' ')
                kdtree = KDTree(pos, leafsize=10)
                np1 = NP1(pos, edgesrc, edgetgt,kdtree)
            else:
                np1 = np.nan
            sums += np1     
        mean = sums/repetitions
        values[graphidx,i] = mean
    graphidx+=1
df_np1 = pd.DataFrame(index = graphs[:idx_l])
for i in range(len(methods)):
    df_np1[methods[i]] = values[:,i]
df_np1.to_csv("./results/Approx_NP1.csv")

In [8]:
df_np1[:idx_a].mean()

Exact        0.598521
BH           0.598745
RVS          0.467575
ibFFT-CPU    0.598252
ibFFT-GPU    0.597003
dtype: float64

In [9]:
from numba_kdtree import KDTree

repetitions = 5
values = np.zeros((idx_l,len(methods)))
graphidx = 0
# for graphname in (graphs): # all graphs
for graphname in (graphs[:idx_l]):
    filename = "../data/" + graphname + ".mtx"
    graph,edgesrc,edgetgt,_ = readgraph(filename,False)
    for i in range(len(methods)):
        sums = 0
        for k in range(repetitions):
            posfile = "../layout_results/" + dirs[i] + "/" + graphname + "." + methods[i] + "_" + str(k)
            if os.path.isfile(posfile):
                if methods[i] == "DRGraph":
                    pos = np.loadtxt(posfile,delimiter=' ',skiprows=1)
                    ## The first line of DRGraph's output is the description of the number of rows and columns
                else:
                    pos = np.loadtxt(posfile,delimiter=' ')
                kdtree = KDTree(pos, leafsize=10)
                np2 = NP2(pos, edgesrc, edgetgt,kdtree)
            else:
                np2 = np.nan
            sums += np2     
        mean = sums/repetitions
        values[graphidx,i] = mean
    graphidx+=1
df_np2 = pd.DataFrame(index = graphs[:idx_l])
for i in range(len(methods)):
    df_np2[methods[i]] = values[:,i]
df_np2.to_csv("./results/Approx_NP2.csv")

In [10]:
df_np2[:idx_a].mean()

Exact        0.625307
BH           0.625391
RVS          0.548972
ibFFT-CPU    0.628405
ibFFT-GPU    0.629298
dtype: float64

In [11]:
from numba_kdtree import KDTree

repetitions = 5
values = np.zeros((idx_l,len(methods)))
graphidx = 0
for graphname in (graphs[:idx_l]):
    filename = "../data/" + graphname + ".mtx"
    graph,edgesrc,edgetgt,_ = readgraph(filename,False)
    for i in range(len(methods)):
        sums = 0
        for k in range(repetitions):
            posfile = "../layout_results/" + dirs[i] + "/" + graphname + "." + methods[i] + "_" + str(k)
            if os.path.isfile(posfile):
                if methods[i] == "DRGraph":
                    pos = np.loadtxt(posfile,delimiter=' ',skiprows=1)
                    ## The first line of DRGraph's output is the description of the number of rows and columns
                else:
                    pos = np.loadtxt(posfile,delimiter=' ')
                kdtree = KDTree(pos, leafsize=10)
                ec_n,ec = EC(pos,graph,edgesrc)
            else:
                ec = np.nan
            sums += ec     
        mean = sums/repetitions
        values[graphidx,i] = mean
    graphidx+=1
df_ec = pd.DataFrame(index = graphs[:idx_l])
for i in range(len(methods)):
    df_ec[methods[i]] = values[:,i]
df_ec.to_csv("./results/Approx_EC.csv")

In [12]:
df_ec[:idx_a].mean()

Exact        0.952246
BH           0.951018
RVS          0.943950
ibFFT-CPU    0.952359
ibFFT-GPU    0.952740
dtype: float64

In [13]:
from numba_kdtree import KDTree

repetitions = 5
values = np.zeros((idx_l,len(methods)))
graphidx = 0
for graphname in (graphs[:idx_l]):
    filename = "../data/" + graphname + ".mtx"
    graph,edgesrc,edgetgt,_ = readgraph(filename,False)
    for i in range(len(methods)):
        sums = 0
        for k in range(repetitions):
            posfile = "../layout_results/" + dirs[i] + "/" + graphname + "." + methods[i] + "_" + str(k)
            if os.path.isfile(posfile):
                if methods[i] == "DRGraph":
                    pos = np.loadtxt(posfile,delimiter=' ',skiprows=1)
                    ## The first line of DRGraph's output is the description of the number of rows and columns
                else:
                    pos = np.loadtxt(posfile,delimiter=' ')
                kdtree = KDTree(pos, leafsize=10)
                ma = MA(pos, edgesrc, edgetgt)
            else:
                ma = np.nan
            sums += ma     
        mean = sums/repetitions
        values[graphidx,i] = mean
    graphidx+=1
df_ma = pd.DataFrame(index = graphs[:idx_l])
for i in range(len(methods)):
    df_ma[methods[i]] = values[:,i]
df_ma.to_csv("./results/Approx_MA.csv")

In [14]:
df_ma[:idx_a].mean()

Exact        0.343800
BH           0.344138
RVS          0.259531
ibFFT-CPU    0.342703
ibFFT-GPU    0.341086
dtype: float64